# Housing Prices Competition for Kaggle Learn Users

### 버전1 - XGBRegressor

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')

In [3]:
test = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv')

In [ ]:
# pd.options.display.max_columns=999

In [ ]:
display(train,test)

In [4]:
alldata = pd.concat([train,test])

In [ ]:
# column들끼리의 관계를 사람이 판단해서 유의미한 조합이 있다면 열을 생성 (column interaction, 교호작용)
alldata['old'] = alldata['YrSold']-alldata['YearBuilt']
alldata['overall'] = alldata['OverallQual']+alldata['OverallCond']

In [ ]:
alldata2 = alldata.drop(columns=['Id','SalePrice'])
alldata2

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

c = alldata2.columns[alldata2.dtypes==object]

for i in c:
    alldata2[i] = le.fit_transform(alldata2[i])
    
alldata2

In [ ]:
alldata2 = alldata2.fillna(-1)
alldata2

In [ ]:
train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# rf = RandomForestRegressor(n_jobs=-1)
# rf.fit(train2,train['SalePrice'])
# result = rf.predict(test2)

from xgboost import XGBRegressor
xgb = XGBRegressor(learning_rate=0.1)
xgb.fit(train2,train['SalePrice'])
result = xgb.predict(test2)

In [22]:
sub = pd.read_csv('/kaggle/input/home-data-for-ml-course/sample_submission.csv')
sub

In [ ]:
sub['SalePrice'] = result
sub.to_csv('sub1.csv',index=0)

### 버전2 - Ridge

In [7]:
alldata

In [6]:
alldata.corr()['SalePrice'].sort_values(ascending=False)

In [8]:
# 선형관계가 많아 보이므로 선형 모델 사용

In [9]:
alldata2 = alldata.drop(columns=['Id', 'SalePrice'])
alldata3 = pd.get_dummies(alldata2)
# 선형 모델 쓸거니까 one hot encoding 사용

In [10]:
alldata3 = alldata3.fillna(-1)
# scaling 전에 결측치 처리

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
alldata4 = scaler.fit_transform(alldata3)
# 선형 모델 쓸거면 수치 범위 비슷하게 맞춰줘야 하므로 scaling

In [14]:
alldata4 = pd.DataFrame(alldata4, columns=alldata3.columns)
# scaler 사용하면 array 되어서 데이터프레임으로 다시 만들어 줌

In [15]:
train2 = alldata4[:len(train)]
test2 = alldata4[len(train):]

In [18]:
from sklearn. linear_model import Ridge

In [19]:
# 데이터 별로 없으니까 cross validation으로 최적의 옵션 찾기
from sklearn.model_selection import cross_val_score
for i in [10, 20, 30, 50, 100, 200, 300, 500, 700, 1000]:
    r_cv = Ridge(alpha=i)
    print(-cross_val_score(r_cv,train2,train['SalePrice'],n_jobs=-1,scoring='neg_mean_absolute_error', cv=10).mean())  # cv: k값. 몇 개로 나누는지.

In [20]:
r = Ridge(alpha=300)  # alpha: 규제 정도 옵션. 가장 작은 값 나온 300 선택
r.fit(train2, np.log(train['SalePrice']))
result = r.predict(test2)

In [23]:
sub['SalePrice'] = np.exp(result)
sub.to_csv('sub2.csv', index=0)